

---

# **Chapter 2: Core Design Principles (The Pillars)**

## **Opening Context**
Design patterns are not applied in a vacuum. They exist to solve problems created by violating fundamental design principles. Before you can effectively wield a Factory or Strategy pattern, you must understand **why** rigidity and fragility occur in code. The SOLID principles—coined by Robert C. Martin (Uncle Bob)—are the guardrails that keep object-oriented systems maintainable. Patterns are the tactical implementations; SOLID is the strategy.

---

## **2.1 SOLID Principles in Depth**

### **Overview**
SOLID is an acronym for five design principles intended to make software designs more understandable, flexible, and maintainable. These principles guide when to use patterns and warn when pattern abuse creates complexity.

---

### **2.1.1 Single Responsibility Principle (SRP)**
**Definition**: *A class should have only one reason to change.*

**The Core Idea**: If a class handles multiple responsibilities (e.g., calculating prices AND formatting invoices AND sending emails), changes to any one responsibility force the class to change, increasing the risk of breaking unrelated functionality.

**Pattern Connection**: SRP drives the use of **Facade**, **Mediator**, and **Command** patterns to separate concerns.

#### **Code Example: Violating vs. Obeying SRP**

```typescript
// ❌ VIOLATION: Multiple reasons to change
class Employee {
    constructor(
        private name: string,
        private salary: number
    ) {}
    
    // Responsibility 1: Data management
    getName(): string { return this.name; }
    setSalary(salary: number): void { this.salary = salary; }
    
    // Responsibility 2: Business logic
    calculatePay(): number { return this.salary * 0.8; } // Tax deduction
    
    // Responsibility 3: Persistence
    saveToDatabase(): void {
        console.log(`Saving ${this.name} to DB...`);
        // Database logic here
    }
    
    // Responsibility 4: Reporting
    generateReport(): string {
        return `Employee Report: ${this.name}, Salary: ${this.salary}`;
    }
    
    // Responsibility 5: Notification
    sendEmail(): void {
        console.log(`Sending email to ${this.name}...`);
    }
}
```

**Why This Violates SRP**:
- Database schema changes force modification (Reason 1)
- Tax calculation rules change force modification (Reason 2)  
- Email protocol changes force modification (Reason 3)
- Report format changes force modification (Reason 4)

**Refactored Solution**:

```typescript
// ✅ SRP COMPLIANT: Separated concerns

// Responsibility: Data entity (anemic model)
class Employee {
    constructor(
        private name: string,
        private salary: number
    ) {}
    
    getName(): string { return this.name; }
    getSalary(): number { return this.salary; }
    setSalary(salary: number): void { this.salary = salary; }
}

// Responsibility: Business logic
class PayCalculator {
    calculateNetSalary(employee: Employee): number {
        return employee.getSalary() * 0.8; // Tax logic isolated
    }
}

// Responsibility: Persistence
class EmployeeRepository {
    save(employee: Employee): void {
        console.log(`Persisting employee: ${employee.getName()}`);
        // Database logic isolated
    }
}

// Responsibility: Reporting
class EmployeeReportGenerator {
    generate(employee: Employee): string {
        return `Report: ${employee.getName()} - $${employee.getSalary()}`;
    }
}

// Responsibility: Communication
class NotificationService {
    sendSalaryNotification(employee: Employee): void {
        console.log(`Notifying ${employee.getName()} of salary deposit`);
    }
}

// Orchestrator (optional - could be a Facade pattern)
class EmployeeService {
    constructor(
        private payCalculator: PayCalculator,
        private repository: EmployeeRepository,
        private notifier: NotificationService
    ) {}
    
    processPayroll(employee: Employee): void {
        const netPay = this.payCalculator.calculateNetSalary(employee);
        this.repository.save(employee);
        this.notifier.sendSalaryNotification(employee);
    }
}
```

**Explanation**:
- **Employee**: Pure data structure (DTO/Entity). Only changes if employee attributes change.
- **PayCalculator**: Isolated tax/business logic. Changes only when calculation rules change.
- **EmployeeRepository**: Database gateway. Changes only when storage mechanism changes.
- **NotificationService**: Communication logic. Changes only when messaging protocols change.

**Trade-off**: More classes to manage, but each can evolve independently. This aligns with the **Strategy** pattern (different calculation strategies) and **Repository** pattern (abstracting data access).

---

### **2.1.2 Open/Closed Principle (OCP)**
**Definition**: *Software entities (classes, modules, functions) should be open for extension but closed for modification.*

**The Core Idea**: You should be able to add new functionality without changing existing, working code. This prevents regression bugs in tested code.

**Pattern Connection**: OCP is the primary driver for **Strategy**, **Decorator**, **Template Method**, and **Observer** patterns.

#### **Code Example: The Extension Problem**

```typescript
// ❌ VIOLATION: Modifying existing code for new features
class AreaCalculator {
    calculate(shape: any): number {
        if (shape.type === 'circle') {
            return Math.PI * shape.radius ** 2;
        } else if (shape.type === 'rectangle') {
            return shape.width * shape.height;
        }
        // ❌ Must modify this method to add Triangle!
        // ❌ Must modify this method to add Polygon!
        // ❌ Every new shape risks breaking existing calculations
        return 0;
    }
}
```

**The Problem**: Adding a `Triangle` requires editing `AreaCalculator`, violating OCP and risking breaking Circle/Rectangle calculations.

**Refactored Solution Using Strategy Pattern**:

```typescript
// ✅ OCP COMPLIANT: Extension via new classes, not modification

// 1. Abstraction (Closed for modification)
interface Shape {
    calculateArea(): number;
}

// 2. Concrete implementations (Open for extension)
class Circle implements Shape {
    constructor(private radius: number) {}
    
    calculateArea(): number {
        return Math.PI * this.radius ** 2;
    }
}

class Rectangle implements Shape {
    constructor(
        private width: number,
        private height: number
    ) {}
    
    calculateArea(): number {
        return this.width * this.height;
    }
}

// 3. New shapes added WITHOUT touching existing code
class Triangle implements Shape {
    constructor(
        private base: number,
        private height: number
    ) {}
    
    calculateArea(): number {
        return 0.5 * this.base * this.height;
    }
}

// 4. Calculator remains unchanged regardless of new shapes
class AreaCalculator {
    calculate(shape: Shape): number {
        // Polymorphism handles dispatch - no if/else chains
        return shape.calculateArea();
    }
    
    calculateTotal(shapes: Shape[]): number {
        return shapes.reduce((sum, shape) => sum + shape.calculateArea(), 0);
    }
}

// Usage
const calculator = new AreaCalculator();
const shapes: Shape[] = [
    new Circle(5),
    new Rectangle(4, 6),
    new Triangle(3, 4) // Added later, no changes to AreaCalculator
];

console.log(calculator.calculateTotal(shapes));
```

**Explanation**:
- **Interface `Shape`**: Defines the contract. Stable, rarely changes.
- **Concrete Classes**: Each shape encapsulates its own area algorithm.
- **AreaCalculator**: Closed for modification. The `calculate` method never changes, yet supports new shapes via polymorphism.

**Industry Note**: This is the foundation of plugin architectures (Webpack loaders, Babel plugins, VS Code extensions).

---

### **2.1.3 Liskov Substitution Principle (LSP)**
**Definition**: *Objects of a superclass shall be replaceable with objects of a subclass without affecting the correctness of the program.*

**The Core Idea**: Inheritance is an "is-a" relationship. If `Square` inherits from `Rectangle`, then `Square` must behave exactly like a `Rectangle` in all contexts, or the inheritance is wrong.

**Pattern Connection**: Violating LSP breaks **Template Method** and **Factory Method** patterns that rely on substitutable subclasses.

#### **Code Example: The Classic Rectangle/Square Violation**

```typescript
// ❌ VIOLATION: Square cannot substitute Rectangle

class Rectangle {
    constructor(
        protected width: number,
        protected height: number
    ) {}
    
    setWidth(width: number): void {
        this.width = width;
    }
    
    setHeight(height: number): void {
        this.height = height;
    }
    
    getArea(): number {
        return this.width * this.height;
    }
}

// Mathematical square "is-a" rectangle, but behaviorally it is not
class Square extends Rectangle {
    constructor(side: number) {
        super(side, side);
    }
    
    // Violation: Changing width MUST change height for Square
    setWidth(width: number): void {
        this.width = width;
        this.height = width; // Side effect!
    }
    
    setHeight(height: number): void {
        this.width = height; // Side effect!
        this.height = height;
    }
}

// Client code expecting Rectangle behavior
function resizeRectangle(rect: Rectangle): void {
    rect.setWidth(5);
    rect.setHeight(4);
    console.log(`Expected area: 20, Actual: ${rect.getArea()}`);
}

// Usage
const rect = new Rectangle(2, 2);
resizeRectangle(rect); // ✅ Expected area: 20, Actual: 20

const square = new Square(2);
resizeRectangle(square); // ❌ Expected area: 20, Actual: 16 (4x4)
```

**The Problem**: `Square` strengthened the preconditions (width must equal height), violating substitutability. The client code broke when given a subclass.

**Refactored Solution: Composition over Inheritance**

```typescript
// ✅ LSP COMPLIANT: No false inheritance hierarchy

interface Shape {
    getArea(): number;
}

class Rectangle implements Shape {
    constructor(
        private width: number,
        private height: number
    ) {}
    
    setWidth(width: number): void { this.width = width; }
    setHeight(height: number): void { this.height = height; }
    getArea(): number { return this.width * this.height; }
}

class Square implements Shape {
    private side: number;
    
    constructor(side: number) {
        this.side = side;
    }
    
    setSide(side: number): void { this.side = side; }
    getArea(): number { return this.side ** 2; }
}

// Factory for creating appropriate shape (Factory Pattern)
class ShapeFactory {
    static createRectangle(width: number, height: number): Shape {
        if (width === height) {
            return new Square(width);
        }
        return new Rectangle(width, height);
    }
}
```

**Explanation**:
- Both implement `Shape` interface, but neither inherits from the other.
- `Square` has its own contract (`setSide`) rather than violating `Rectangle`'s contract.
- Client code using `Shape` interface works with both, but specific `Rectangle` logic won't accidentally receive a `Square`.

**Red Flags for LSP Violations**:
- Empty method overrides (`// not applicable for this subclass`)
- Runtime type checking (`if (obj instanceof Subclass)`)
- Overridden methods that throw exceptions (`throw new Error("Not supported")`)

---

### **2.1.4 Interface Segregation Principle (ISP)**
**Definition**: *Clients should not be forced to depend on interfaces they do not use.*

**The Core Idea**: Fat interfaces (god interfaces) force implementing classes to provide dummy implementations for methods they don't need. Split large interfaces into smaller, cohesive ones.

**Pattern Connection**: Drives the use of **Adapter** pattern and role interfaces.

#### **Code Example: The Fat Interface Problem**

```typescript
// ❌ VIOLATION: Monolithic interface
interface Worker {
    work(): void;
    eat(): void;
    sleep(): void;
    attendMeeting(): void;
    code(): void;
    manageTeam(): void;
}

// Robot implements Worker but doesn't eat, sleep, or attend meetings!
class Robot implements Worker {
    work(): void { console.log("Working..."); }
    eat(): void { /* Dummy implementation */ }
    sleep(): void { /* Dummy implementation */ }
    attendMeeting(): void { /* Dummy implementation */ }
    code(): void { console.log("Coding..."); }
    manageTeam(): void { throw new Error("Robots don't manage"); }
}

// Human implements everything, but maybe not all humans code or manage
class Human implements Worker {
    work(): void { console.log("Working..."); }
    eat(): void { console.log("Eating..."); }
    sleep(): void { console.log("Sleeping..."); }
    attendMeeting(): void { console.log("Meeting..."); }
    code(): void { /* Not all humans code */ }
    manageTeam(): void { /* Not all humans manage */ }
}
```

**Refactored Solution: Role Interfaces**

```typescript
// ✅ ISP COMPLIANT: Segregated interfaces

interface Workable {
    work(): void;
}

interface Feedable {
    eat(): void;
    sleep(): void;
}

interface MeetingAttendee {
    attendMeeting(): void;
}

interface Coder {
    code(): void;
}

interface Manager {
    manageTeam(): void;
}

// Robot implements only what it needs
class Robot implements Workable, Coder {
    work(): void { console.log("Robot working 24/7"); }
    code(): void { console.log("Robot compiling..."); }
}

// Developer implements relevant interfaces
class Developer implements Workable, Feedable, MeetingAttendee, Coder {
    work(): void { console.log("Developing features"); }
    eat(): void { console.log("Lunch break"); }
    sleep(): void { console.log("Recharging"); }
    attendMeeting(): void { console.log("Standup"); }
    code(): void { console.log("Typing code"); }
}

// Manager implements management interface
class Manager implements Workable, Feedable, MeetingAttendee, Manager {
    work(): void { console.log("Managing project"); }
    eat(): void { console.log("Business lunch"); }
    sleep(): void { console.log("Resting"); }
    attendMeeting(): void { console.log("Planning meeting"); }
    manageTeam(): void { console.log("1-on-1s"); }
}

// Client code depends only on what it needs
class MeetingRoom {
    scheduleMeeting(attendee: MeetingAttendee): void {
        // Depends only on MeetingAttendee, not full Worker interface
        attendee.attendMeeting();
    }
}
```

**Explanation**:
- **Role Interfaces**: `Workable`, `Feedable`, etc., represent specific roles.
- **Robot**: No longer forced to implement `eat()` or `sleep()`.
- **MeetingRoom**: Depends only on `MeetingAttendee`, not caring if the attendee is a Developer, Manager, or Robot (if robots ever attend meetings).

**Connection to Patterns**: This is the **Adapter** pattern's foundation—adapting specific interfaces to client needs.

---

### **2.1.5 Dependency Inversion Principle (DIP)**
**Definition**: 
1. *High-level modules should not depend on low-level modules. Both should depend on abstractions.*
2. *Abstractions should not depend on details. Details should depend on abstractions.*

**The Core Idea**: Traditional procedural programming creates dependency graphs where high-level modules call low-level modules directly. DIP inverts this using abstractions (interfaces).

**Pattern Connection**: The foundation of **Dependency Injection**, **Factory**, **Strategy**, and **Bridge** patterns.

#### **Code Example: Traditional vs. Inverted Dependencies**

```typescript
// ❌ VIOLATION: High-level module depends on low-level details

// Low-level module
class MySQLDatabase {
    connect(): void { console.log("MySQL connected"); }
    query(sql: string): any[] { return []; }
}

// High-level module (Business logic)
class UserService {
    private db: MySQLDatabase; // Concrete dependency!
    
    constructor() {
        this.db = new MySQLDatabase(); // Hard-coded!
    }
    
    getUsers(): any[] {
        this.db.connect();
        return this.db.query("SELECT * FROM users");
    }
}
```

**Problems**:
- Cannot switch to PostgreSQL without modifying `UserService`
- Cannot unit test without real database
- `UserService` knows about SQL (detail), not just data access (abstraction)

**Refactored Solution: Dependency Inversion**

```typescript
// ✅ DIP COMPLIANT: Depend on abstractions

// 1. Abstraction (Interface)
interface Database {
    connect(): void;
    query(query: string): any[];
}

// 2. Low-level modules depend on abstraction
class MySQLDatabase implements Database {
    connect(): void { console.log("MySQL connected"); }
    query(sql: string): any[] { 
        console.log("Executing MySQL query");
        return []; 
    }
}

class PostgreSQLDatabase implements Database {
    connect(): void { console.log("PostgreSQL connected"); }
    query(sql: string): any[] { 
        console.log("Executing PostgreSQL query");
        return []; 
    }
}

class InMemoryDatabase implements Database {
    private data: any[] = [];
    
    connect(): void { console.log("In-memory DB ready"); }
    query(q: string): any[] { 
        console.log("Searching memory");
        return this.data; 
    }
}

// 3. High-level module depends on abstraction
class UserService {
    // Depends on interface, not implementation
    constructor(private db: Database) {}
    
    getUsers(): any[] {
        this.db.connect();
        return this.db.query("SELECT * FROM users");
    }
}

// 4. Composition Root (where concrete choices are made)
class Application {
    static main(): void {
        // Switch implementations without touching UserService
        const mysqlDb: Database = new MySQLDatabase();
        const userService = new UserService(mysqlDb);
        userService.getUsers();
        
        // Testing with mock
        const mockDb: Database = new InMemoryDatabase();
        const testService = new UserService(mockDb);
        testService.getUsers();
    }
}
```

**Explanation**:
- **Database Interface**: The abstraction both sides depend on.
- **Concrete Databases**: Details that depend on the abstraction (they implement it).
- **UserService**: High-level policy that depends only on `Database` interface.
- **Dependency Injection**: Dependencies are "injected" via constructor rather than created internally.

**DIP in Practice**: This is why modern frameworks (Spring, Angular, ASP.NET Core) use Dependency Injection containers.

---

## **2.2 Auxiliary Principles: DRY, KISS, and YAGNI**

While SOLID handles architectural structure, these principles govern implementation decisions.

### **2.2.1 DRY (Don't Repeat Yourself)**
**Definition**: *Every piece of knowledge must have a single, unambiguous, authoritative representation in the system.*

**The Nuance**: DRY is about *knowledge*, not just code. Two functions checking `user.age >= 18` in different contexts might look like duplication, but if they represent different business rules (voting age vs. drinking age), they should remain separate.

**Code Example**:

```typescript
// ❌ WET (Write Everything Twice): Duplicated validation logic
class UserRegistration {
    validateEmail(email: string): boolean {
        const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
        return regex.test(email);
    }
}

class NewsletterService {
    validateEmail(email: string): boolean {
        const regex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/; // Same regex!
        return regex.test(email);
    }
}

// ✅ DRY: Single source of truth
class EmailValidator {
    private static readonly EMAIL_REGEX = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    
    static isValid(email: string): boolean {
        return this.EMAIL_REGEX.test(email);
    }
}

// Both services use the same knowledge representation
class UserRegistration {
    register(email: string): void {
        if (!EmailValidator.isValid(email)) throw new Error("Invalid");
    }
}
```

---

### **2.2.2 KISS (Keep It Simple, Stupid)**
**Definition**: *Most systems work best if they are kept simple rather than made complicated.*

**Application**: Prefer straightforward solutions over clever, abstract ones. If a `switch` statement suffices, don't use a Strategy pattern with XML configuration.

**Code Example**:

```typescript
// ❌ OVER-ENGINEERED: Strategy pattern for 2 static options
interface PaymentStrategy {
    pay(amount: number): void;
}

class CreditCardStrategy implements PaymentStrategy {
    pay(amount: number): void { /* ... */ }
}

class PayPalStrategy implements PaymentStrategy {
    pay(amount: number): void { /* ... */ }
}

class PaymentProcessor {
    constructor(private strategy: PaymentStrategy) {}
    process(amount: number): void {
        this.strategy.pay(amount);
    }
}

// ✅ KISS: Simple conditional when logic is trivial and unlikely to grow
function processPayment(method: 'credit' | 'paypal', amount: number): void {
    if (method === 'credit') {
        // Credit logic
    } else {
        // PayPal logic
    }
}

// Refactor to Strategy ONLY when 3+ methods exist or runtime switching is needed
```

---

### **2.2.3 YAGNI (You Aren't Gonna Need It)**
**Definition**: *Don't implement functionality until you actually need it.*

**Context**: This prevents speculative generality. Don't build a plugin system for an application with one fixed requirement. Don't abstract database layers when you're committed to one database for the foreseeable future.

**Pattern Connection**: Related to **Simple Factory** vs. **Abstract Factory**. Use Simple Factory until you actually need the flexibility of Abstract Factory.

---

## **2.3 The "Composition over Inheritance" Debate**

### **The Problem with Inheritance**
Inheritance creates the tightest coupling in OOP. A subclass is bound to its superclass's implementation details, breaking encapsulation.

**Issues**:
1.  **Fragile Base Class Problem**: Changes to parent break children
2.  **Hierarchy Rigidity**: Class hierarchies are fixed at compile time
3.  **Multiple Inheritance Complexity**: Diamond problem in C++, limited in Java/TS

### **Composition**
Composition involves containing instances of other classes rather than inheriting from them. It provides:
- **Flexibility**: Behaviors can be changed at runtime
- **Loose Coupling**: Components interact through interfaces
- **Testability**: Easy to mock composed objects

#### **Code Example: Inheritance vs. Composition**

```typescript
// ❌ INHERITANCE: Rigid hierarchy

class Animal {
    move(): void {
        console.log("Moving...");
    }
}

class Bird extends Animal {
    fly(): void {
        console.log("Flying...");
    }
}

class Penguin extends Bird {
    // Problem: Penguins can't fly!
    fly(): void {
        throw new Error("Penguins can't fly!"); // LSP violation
    }
}

// ✅ COMPOSITION: Flexible behavior assembly

interface MovementStrategy {
    move(): void;
}

class WalkingStrategy implements MovementStrategy {
    move(): void { console.log("Walking..."); }
}

class FlyingStrategy implements MovementStrategy {
    move(): void { console.log("Flying..."); }
}

class SwimmingStrategy implements MovementStrategy {
    move(): void { console.log("Swimming..."); }
}

class Animal {
    // Composed behavior, not inherited
    constructor(private movement: MovementStrategy) {}
    
    move(): void {
        this.movement.move();
    }
    
    // Runtime behavior change (State pattern)
    setMovementStrategy(strategy: MovementStrategy): void {
        // this.movement = strategy;
    }
}

// Usage
const sparrow = new Animal(new FlyingStrategy());
const penguin = new Animal(new SwimmingStrategy());
const dog = new Animal(new WalkingStrategy());

// Duck can swim AND walk (multiple behaviors via composition)
class Duck {
    constructor(
        private primaryMovement: MovementStrategy,
        private secondaryMovement: MovementStrategy
    ) {}
    
    move(): void {
        this.primaryMovement.move();
    }
}
```

**Explanation**:
- **Inheritance**: `Penguin` is forced to override `fly()` with an exception, violating LSP.
- **Composition**: `Animal` delegates movement to a `MovementStrategy`. `Penguin` simply receives a `SwimmingStrategy`. No broken inheritance.
- **Runtime Flexibility**: An injured bird could switch from `FlyingStrategy` to `WalkingStrategy` at runtime—impossible with inheritance.

**The Principle**: *"Favor composition over inheritance"* (GoF).

**When to Use Inheritance**:
- True taxonomic "is-a" relationships (e.g., `SavingsAccount` is-a `BankAccount`)
- When overriding provides true specialization, not negation
- Framework extension points designed for inheritance (Template Method pattern)

---

## **Chapter Summary**
In this chapter, we established the foundational principles:
1.  **SRP**: One class, one reason to change. Drives separation of concerns.
2.  **OCP**: Extend behavior without modifying code. Enables plugin architectures.
3.  **LSP**: Subclasses must be substitutable. Prevents broken inheritance hierarchies.
4.  **ISP**: Split fat interfaces. Reduces coupling to unused methods.
5.  **DIP**: Depend on abstractions. The foundation of dependency injection.
6.  **DRY/KISS/YAGNI**: Practical guidelines preventing over-engineering.
7.  **Composition over Inheritance**: Prefer assembling behaviors over inheriting them.

**Key Insight**: Patterns are the *how*; SOLID is the *why*. When choosing between patterns, ask: *"Which solution best satisfies SOLID principles for my specific context?"*

---

## **Next Chapter Preview**
**Chapter 3: Documentation and Communication**

Now that we understand the principles and vocabulary of patterns, we must learn to communicate them. Chapter 3 covers **UML Class Diagrams** and **Sequence Diagrams**—the industry standard visual languages for documenting architecture. We will learn to read and create diagrams that accurately represent the static structure (classes, relationships) and dynamic behavior (object interactions over time) of pattern implementations. This skill is essential for architecture reviews, technical documentation, and onboarding developers to pattern-based codebases.

---

